In [ ]:
import os
import scanpy as sc
import metacells as mc
import pandas as pd
import numpy as np
from pathlib import Path
import scipy.sparse as sp
import matplotlib.pyplot as plt
import seaborn as sns
import metacells.tools.high as mct

/Users/guyshani/Documents/PHD/Aim_2/process_sc/.venv_metacells/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [82]:
import traceback

In [54]:
class SCProcessor:
    def __init__(self, base_directories, output_dir="processed_data", random_seed=42,
                 target_metacell_size=100, target_metacell_umis=160000,
                 min_genes=200, max_genes=6000, max_mt_percent=20):
        self.base_directories = base_directories
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(exist_ok=True)
        self.random_seed = random_seed
        self.target_metacell_size = target_metacell_size
        self.target_metacell_umis = target_metacell_umis
        self.min_genes = min_genes
        self.max_genes = max_genes
        self.max_mt_percent = max_mt_percent
        
        # Configure scanpy settings
        sc.settings.verbosity = 3
        sc.settings.set_figure_params(dpi=80)
        
    def load_10x_data(self, data_dir):
        try:
            adata = sc.read_10x_mtx(
                data_dir,
                var_names='gene_symbols',
                cache=True
            )
            adata.obs['dataset'] = Path(data_dir).parent.name
            return adata
        except Exception as e:
            print(f"Error loading data from {data_dir}: {e}")
            return None

    def process_dataset(self, adata):
        """Basic preprocessing using scanpy"""
        # First, annotate mitochondrial genes
        adata.var['mt'] = adata.var_names.str.startswith('mt-')  # Mouse mitochondrial genes
        
        # Calculate QC metrics
        sc.pp.calculate_qc_metrics(
            adata,
            qc_vars=['mt'],
            percent_top=None,
            log1p=False,
            inplace=True
        )
        
        # Basic filtering using instance parameters
        print(f"Before filtering: {adata.shape[0]} cells")
        sc.pp.filter_cells(adata, min_genes=self.min_genes)
        sc.pp.filter_cells(adata, max_genes=self.max_genes)
        adata = adata[adata.obs.pct_counts_mt < self.max_mt_percent]
        print(f"After filtering: {adata.shape[0]} cells")
        
        # Normalize data
        sc.pp.normalize_total(adata, target_sum=1e4)
        sc.pp.log1p(adata)
        
        return adata
        
    def get_mouse_cell_cycle_genes(self):
        """Return a list of mouse cell cycle genes to be treated as lateral genes"""
        return [
            'Mki67',   # Instead of MKI67
            'Pcna',    # Instead of PCNA
            'Mcm2', 'Mcm3', 'Mcm4', 'Mcm5', 'Mcm6', 'Mcm7',  # Instead of MCM[2-7]
            'Cdc20',   # Instead of CDC20
            'Cdk1',    # Instead of CDK1
            'Top2a',   # Additional mouse cell cycle gene
            'Ccnb1',   # Additional mouse cell cycle gene
            'Ccna2',   # Additional mouse cell cycle gene
            'Aurka',   # Additional mouse cell cycle gene
            'Birc5',   # Additional mouse cell cycle gene
            'Cdc25c'   # Additional mouse cell cycle gene
        ]
        
    def run_metacell_analysis(self, adata):
        print("Starting metacell analysis...")
        
        # Convert to float32 to ensure compatibility
        adata.X = adata.X.astype('float32')
        
        # Mark cell cycle genes as lateral genes
        cell_cycle_genes = self.get_mouse_cell_cycle_genes()
        adata.var['lateral_gene'] = adata.var_names.isin(cell_cycle_genes)
        
        # Mark mitochondrial and ribosomal genes as excluded
        adata.var['excluded_gene'] = (
            adata.var_names.str.startswith('mt-') |  # Mouse mitochondrial genes
            adata.var_names.str.startswith(('Rps', 'Rpl'))  # Mouse ribosomal genes
        )
        
        print("Running divide and conquer pipeline...")
        mc.pl.divide_and_conquer_pipeline(
            adata,
            target_metacell_size=self.target_metacell_size,
            target_metacell_umis=self.target_metacell_umis,
            random_seed=self.random_seed
        )
        
        # Collect metacells
        print("Collecting metacells...")
        metacells = mc.pl.collect_metacells(
            adata,
            random_seed=self.random_seed
        )
        
        # Compute our own QC metrics
        print("Computing QC metrics...")
        
        # Calculate metacell sizes
        metacell_sizes = metacells.obs['grouped']
        print(f"\nMetacell size statistics:")
        print(f"- Total metacells: {len(metacell_sizes)}")
        print(f"- Mean cells per metacell: {metacell_sizes.mean():.1f}")
        print(f"- Median cells per metacell: {metacell_sizes.median():.1f}")
        
        # Calculate UMI statistics
        umi_counts = metacells.obs['total_umis']
        print(f"\nUMI statistics:")
        print(f"- Mean UMIs per metacell: {umi_counts.mean():.1f}")
        print(f"- Median UMIs per metacell: {umi_counts.median():.1f}")
        
        # Gene statistics - convert sparse matrix to numpy array
        if sp.issparse(metacells.X):
            n_expressed = (metacells.X > 0).sum(axis=1).A1  # Convert to 1D numpy array
        else:
            n_expressed = (metacells.X > 0).sum(axis=1)
            
        print(f"\nGene statistics:")
        print(f"- Mean expressed genes per metacell: {n_expressed.mean():.1f}")
        print(f"- Median expressed genes per metacell: {np.median(n_expressed):.1f}")
        
        # Store QC metrics in metacells object
        metacells.obs['n_expressed_genes'] = n_expressed
        
        # Try to compute UMAP if possible
        try:
            print("\nComputing UMAP projection...")
            sc.pp.neighbors(metacells)
            sc.tl.umap(metacells)
        except Exception as e:
            print(f"Could not compute UMAP: {str(e)}")
        
        print("\nMetacell analysis complete.")
        return metacells
        
    def run_full_pipeline(self):
        all_data = []
        
        # Load and process each dataset
        for base_dir in self.base_directories:
            print(f"\nProcessing dataset in {base_dir}")
            
            if not Path(base_dir).exists():
                print(f"Directory not found: {base_dir}")
                continue
                
            adata = self.load_10x_data(base_dir)
            if adata is None:
                continue
                
            adata = self.process_dataset(adata)
            all_data.append(adata)
        
        if not all_data:
            raise ValueError("No datasets were successfully processed")
            
        print("\nCombining all datasets...")
        combined_data = all_data[0].concatenate(
            all_data[1:],
            join='outer',
            batch_key='batch'
        )
        
        print("\nRunning metacell analysis...")
        metacells = self.run_metacell_analysis(combined_data)
        
        print("\nPrepping data for saving...")
        
        def clean_anndata(adata):
            """Clean AnnData object for saving"""
            # Clean var dataframe
            for col in adata.var.columns:
                try:
                    # Convert all problematic columns to string
                    if adata.var[col].dtype == bool or 'object' in str(adata.var[col].dtype):
                        adata.var[col] = adata.var[col].astype(str)
                    # Remove problematic characters from column names
                    if '-' in col:
                        new_col = col.replace('-', '_')
                        adata.var.rename(columns={col: new_col}, inplace=True)
                except Exception as e:
                    print(f"Warning: Could not process column {col}: {str(e)}")
            
            # Clean obs dataframe
            for col in adata.obs.columns:
                try:
                    if adata.obs[col].dtype == bool or 'object' in str(adata.obs[col].dtype):
                        adata.obs[col] = adata.obs[col].astype(str)
                except Exception as e:
                    print(f"Warning: Could not process column {col}: {str(e)}")
            
            return adata
        
        try:
            print("Cleaning combined data...")
            combined_data = clean_anndata(combined_data)
            
            print("Cleaning metacells data...")
            metacells = clean_anndata(metacells)
            
            print("Saving results...")
            combined_data.write(
                self.output_dir / 'combined_data.h5ad',
                compression='gzip'
            )
            metacells.write(
                self.output_dir / 'metacells.h5ad',
                compression='gzip'
            )
            print("Data saved successfully!")
            
        except Exception as e:
            print(f"Error during saving: {str(e)}")
            # Try to save in a different format if h5ad fails
            try:
                print("Attempting to save as compressed npz...")
                combined_data.write_csvs(self.output_dir / 'combined_data_csvs')
                metacells.write_csvs(self.output_dir / 'metacells_csvs')
                print("Data saved in CSV format as fallback")
            except Exception as e2:
                print(f"Could not save data in any format: {str(e2)}")
                raise
        
        return combined_data, metacells

In [55]:
if __name__ == "__main__":
    # List of directories containing 10X data
    data_dirs = [
        "/Users/guyshani/Documents/PHD/Aim_2/10x_data_mouse/SC3_v3_NextGem_DI_CellPlex_Mouse_PBMC_10K_PBMCs_mouse_1_count_sample_feature_bc_matrix/",
        "/Users/guyshani/Documents/PHD/Aim_2/10x_data_mouse/SC5v2_mousePBMCs_1Kcells_Connect_single_channel_SC5v2_mousePBMCs_1Kcells_Connect_single_channel_count_sample_feature_bc_matrix/",
        "/Users/guyshani/Documents/PHD/Aim_2/10x_data_mouse/sc5p_v2_mm_balbc_T_1k_multi_5gex_t_count_raw_feature_bc_matrix/",
        "/Users/guyshani/Documents/PHD/Aim_2/10x_data_mouse/vdj_nextgem_mm_pbmc4_raw_feature_bc_matrix/",
        "/Users/guyshani/Documents/PHD/Aim_2/10x_data_mouse/vdj_v1_mm_balbc_pbmc_5gex_raw_feature_bc_matrix/",
        "/Users/guyshani/Documents/PHD/Aim_2/10x_data_mouse/vdj_v1_mm_c57bl6_pbmc_5gex_raw_feature_bc_matrix/",
        "/Users/guyshani/Documents/PHD/Aim_2/10x_data_mouse/vdj_v1_mm_pbmc4_raw_feature_bc_matrix/"
    ]

    # Pipeline parameters
    params = {
        "output_dir": "/Users/guyshani/Documents/PHD/Aim_2/process_sc",
        "random_seed": 42,
        # Metacell parameters
        "target_metacell_size": 100,    # Number of cells per metacell
        "target_metacell_umis": 160000, # Target UMIs per metacell
        # QC parameters
        "min_genes": 200,               # Minimum genes per cell
        "max_genes": 6000,              # Maximum genes per cell
        "max_mt_percent": 20            # Maximum percent mitochondrial
    }
    
    # Initialize and run pipeline
    processor = SCProcessor(
        base_directories=data_dirs,
        **params
    )
    
    combined_data, metacells = processor.run_full_pipeline()




Processing dataset in /Users/guyshani/Documents/PHD/Aim_2/10x_data_mouse/SC3_v3_NextGem_DI_CellPlex_Mouse_PBMC_10K_PBMCs_mouse_1_count_sample_feature_bc_matrix/
... reading from cache file cache/Users-guyshani-Documents-PHD-Aim_2-10x_data_mouse-SC3_v3_NextGem_DI_CellPlex_Mouse_PBMC_10K_PBMCs_mouse_1_count_sample_feature_bc_matrix-matrix.h5ad
Before filtering: 4598 cells
filtered out 38 cells that have less than 200 genes expressed
filtered out 3 cells that have more than 6000 genes expressed
After filtering: 4306 cells
normalizing counts per cell
    finished (0:00:00)

Processing dataset in /Users/guyshani/Documents/PHD/Aim_2/10x_data_mouse/SC5v2_mousePBMCs_1Kcells_Connect_single_channel_SC5v2_mousePBMCs_1Kcells_Connect_single_channel_count_sample_feature_bc_matrix/
... reading from cache file cache/Users-guyshani-Documents-PHD-Aim_2-10x_data_mouse-SC5v2_mousePBMCs_1Kcells_Connect_single_channel_SC5v2_mousePBMCs_1Kcells_Connect_single_channel_count_sample_feature_bc_matrix-matrix.h5a

/Users/guyshani/Documents/PHD/Aim_2/process_sc/.venv_metacells/lib/python3.11/site-packages/scanpy/preprocessing/_normalization.py:207: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


After filtering: 982 cells
normalizing counts per cell
    finished (0:00:00)

Processing dataset in /Users/guyshani/Documents/PHD/Aim_2/10x_data_mouse/sc5p_v2_mm_balbc_T_1k_multi_5gex_t_count_raw_feature_bc_matrix/
... reading from cache file cache/Users-guyshani-Documents-PHD-Aim_2-10x_data_mouse-sc5p_v2_mm_balbc_T_1k_multi_5gex_t_count_raw_feature_bc_matrix-matrix.h5ad


/Users/guyshani/Documents/PHD/Aim_2/process_sc/.venv_metacells/lib/python3.11/site-packages/scanpy/preprocessing/_normalization.py:207: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


Before filtering: 737280 cells
filtered out 736284 cells that have less than 200 genes expressed
After filtering: 982 cells
normalizing counts per cell
    finished (0:00:00)

Processing dataset in /Users/guyshani/Documents/PHD/Aim_2/10x_data_mouse/vdj_nextgem_mm_pbmc4_raw_feature_bc_matrix/
... reading from cache file cache/Users-guyshani-Documents-PHD-Aim_2-10x_data_mouse-vdj_nextgem_mm_pbmc4_raw_feature_bc_matrix-matrix.h5ad


...

set unnamed.var[rare_gene]: 0 true (0%) out of 33502 bools
set unnamed.var[rare_gene_module]: 33502 int32 elements with all outliers (100%)
set unnamed.obs[cells_rare_gene_module]: 43550 int32 elements with all outliers (100%)
set unnamed.obs[rare_cell]: 0 true (0%) out of 43550 bools
set unnamed.var[selected_gene]: 4626 true (13.81%) out of 33502 bools
set unnamed.obs[metacell]: 43550 int32s
set unnamed.obs[dissolved]: 5 true (0.01148%) out of 43550 bools
set unnamed.obs[metacell_level]: 43550 int32s


set metacells.obs[grouped]: 576 int64s
set metacells.obs[total_umis]: 576 int64s
set metacells.layers[total_umis]: ndarray 576 X 33502 float32s
set metacells.obs[__zeros_downsample_umis]: 576 int64s
set metacells.layers[zeros]: ndarray 576 X 33502 int32s
set unnamed.obs[metacell_name]: 43550 <U8s
set metacells.var[gene_ids-0]: 33502 objects
set metacells.var[feature_types-0]: 33502 objects
set metacells.var[mt-0]: 33502 objects
set metacells.var[n_cells_by_counts-0]: 33502 float64s
set metacells.var[mean_counts-0]: 33502 float32s
set metacells.var[pct_dropout_by_counts-0]: 33502 float64s
set metacells.var[total_counts-0]: 33502 float32s
set metacells.var[gene_ids-1]: 33502 objects
set metacells.var[feature_types-1]: 33502 objects
set metacells.var[mt-1]: 33502 objects
set metacells.var[n_cells_by_counts-1]: 33502 float64s
set metacells.var[mean_counts-1]: 33502 float32s
set metacells.var[pct_dropout_by_counts-1]: 33502 float64s
set metacells.var[total_counts-1]: 33502 float32s
set meta

Computing QC metrics...

Metacell size statistics:
- Total metacells: 576
- Mean cells per metacell: 75.6
- Median cells per metacell: 73.0

UMI statistics:
- Mean UMIs per metacell: 176612.7
- Median UMIs per metacell: 169497.0

Gene statistics:
- Mean expressed genes per metacell: 9926.7
- Median expressed genes per metacell: 9859.0

Computing UMAP projection...
computing neighbors
computing PCA
    with n_comps=50


/Users/guyshani/Documents/PHD/Aim_2/process_sc/.venv_metacells/lib/python3.11/site-packages/scanpy/tools/_utils.py:41: UserWarning: You’re trying to run this on 33502 dimensions of `.X`, if you really want this, set `use_rep='X'`.
         Falling back to preprocessing with `sc.pp.pca` and default params.
  warnings.warn(


    finished (0:00:02)
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:02)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm)
    'umap', UMAP parameters (adata.uns) (0:00:00)

Metacell analysis complete.

Prepping data for saving...
Cleaning combined data...
Cleaning metacells data...
Saving results...
Data saved successfully!


In [85]:
def compute_umap(metacells, seed=42):
    """Helper function to compute UMAP for metacells"""
    print("Computing UMAP...")
    
    try:
        # Use scanpy for UMAP computation instead
        import scanpy as sc
        
        # Normalize if needed
        if 'log1p' not in metacells.uns:
            sc.pp.normalize_total(metacells)
            sc.pp.log1p(metacells)
        
        # Find highly variable genes
        sc.pp.highly_variable_genes(metacells, n_top_genes=2000)
        
        # Compute PCA
        sc.pp.pca(metacells, use_highly_variable=True, random_state=seed)
        
        # Compute neighbors
        sc.pp.neighbors(metacells, random_state=seed)
        
        # Compute UMAP
        sc.tl.umap(metacells, random_state=seed)
        
        return True
            
    except Exception as e:
        print(f"Could not compute UMAP: {str(e)}")
        return False
    
def compute_metacell_umap(metacells, seed=42):
    """Helper function to compute UMAP for metacells"""
    

    print("Finding marker genes...")
    
    try:
        # Convert string columns to proper boolean masks
        for col in metacells.var.columns:
            if metacells.var[col].dtype == object:
                metacells.var[col] = metacells.var[col].astype(str) == 'True'
        
        # Find marker genes using the tools module
        mct.find_metacells_marker_genes(
            metacells
            #min_gene_range_fold=2,
            #min_max_gene_fraction=0.2
        )
        
        print("Computing UMAP using marker genes...")
        # Get marker genes
        if 'marker_gene' in metacells.var:
            marker_genes = metacells.var_names[metacells.var['marker_gene']].tolist()
            print(f"Found {len(marker_genes)} marker genes")
            
            return True
        else:
            print("No marker genes found")
            return False
            
        mc.pl.compute_umap_by_markers(
            metacells,
            marker_gene_names=marker_genes,
            random_seed=seed
            )
        '''
        mc.pl.compute_knn_by_markers(
            metacells,
            what="__x__",
            marker_gene_names=marker_genes,
            random_seed=seed
        )
        sc.tl.umap(metacells, random_state=seed)
        '''
    except Exception as e:
        print(f"Could not compute UMAP: {str(e)}")
        traceback.print_exc()
        return False

def visualize_metacells(metacells, combined_data, seed=42):
    """
    Create visualizations for metacell analysis results with both UMAP approaches
    """
    # Calculate both UMAPs
    print("Computing metacell marker-based UMAP...")
    has_marker_umap = compute_metacell_umap(metacells, seed)
    
    # Store the marker-based UMAP if it exists
    if has_marker_umap and 'X_umap' in metacells.obsm:
        marker_umap = metacells.obsm['X_umap'].copy()
    else:
        marker_umap = None
    
    print("\nComputing standard scanpy UMAP...")
    has_standard_umap = compute_umap(metacells, seed)
    
    # Create basic plots - now with 6 panels
    fig = plt.figure(figsize=(20, 15))
    gs = plt.GridSpec(2, 3, figure=fig)
    
    # 1. Metacell sizes
    ax1 = fig.add_subplot(gs[0, 0])
    sns.histplot(data=metacells.obs['grouped'], ax=ax1)
    ax1.set_title('Distribution of Metacell Sizes')
    ax1.set_xlabel('Number of Cells per Metacell')
    
    # 2. UMIs per metacell
    ax2 = fig.add_subplot(gs[0, 1])
    sns.histplot(data=metacells.obs['total_umis'], ax=ax2)
    ax2.set_title('Distribution of UMIs per Metacell')
    ax2.set_xlabel('Total UMIs')
    
    # 3. Marker-based UMAP
    ax3 = fig.add_subplot(gs[0, 2])
    if marker_umap is not None:
        ax3.scatter(marker_umap[:, 0], marker_umap[:, 1], alpha=0.6)
        ax3.set_title('Marker-based UMAP')
        ax3.set_xlabel('UMAP1')
        ax3.set_ylabel('UMAP2')
    else:
        ax3.text(0.5, 0.5, 'Marker-based UMAP\nnot available', 
                horizontalalignment='center',
                verticalalignment='center')
        ax3.set_title('Marker-based UMAP (Not Available)')
    
    # 4. Standard UMAP
    ax4 = fig.add_subplot(gs[1, 0])
    if has_standard_umap and 'X_umap' in metacells.obsm:
        ax4.scatter(metacells.obsm['X_umap'][:, 0], 
                   metacells.obsm['X_umap'][:, 1], 
                   alpha=0.6)
        ax4.set_title('Standard UMAP')
        ax4.set_xlabel('UMAP1')
        ax4.set_ylabel('UMAP2')
    else:
        ax4.text(0.5, 0.5, 'Standard UMAP\nnot available', 
                horizontalalignment='center',
                verticalalignment='center')
        ax4.set_title('Standard UMAP (Not Available)')
    
    # 5. Batch distribution in metacells
    ax5 = fig.add_subplot(gs[1, 1:])
    try:
        # First ensure metacell information is in combined_data
        if 'metacell' not in combined_data.obs.columns:
            print("Adding metacell assignments to combined data...")
            metacell_dict = dict(enumerate(metacells.obs.index))
            combined_data.obs['metacell'] = [metacell_dict.get(i, 'None') 
                                           for i in range(len(combined_data.obs))]
        
        batch_counts = combined_data.obs.groupby(['metacell', 'batch'], observed=True).size().unstack(fill_value=0)
        batch_fractions = batch_counts.div(batch_counts.sum(axis=1), axis=0)
        
        # Plot batch distribution
        sns.boxplot(data=batch_fractions, ax=ax5)
        ax5.set_title('Batch Distribution in Metacells')
        
        # Fix x-axis labels
        plt.setp(ax5.xaxis.get_majorticklabels(), rotation=45, ha='right')
        
        # Calculate and print batch mixing statistics
        entropy = -(batch_fractions * np.log2(batch_fractions + 1e-10)).sum(axis=1)
        max_entropy = -np.log2(1/batch_fractions.shape[1])
        mixing_score = entropy / max_entropy
        
        print("\nBatch Mixing Statistics:")
        print(f"Average batch mixing score: {mixing_score.mean():.3f}")
        print(f"Median batch mixing score: {mixing_score.median():.3f}")
        
    except Exception as e:
        print(f"Could not create batch distribution plot: {str(e)}")
        ax5.text(0.5, 0.5, 'Batch distribution not available', 
                horizontalalignment='center',
                verticalalignment='center')
        ax5.set_title('Batch Distribution (Not Available)')
    
    plt.tight_layout()
    
    # Print summary statistics
    print("\nMetacell Summary Statistics:")
    print(f"Total number of metacells: {len(metacells)}")
    print(f"Average cells per metacell: {metacells.obs['grouped'].mean():.1f}")
    print(f"Median cells per metacell: {metacells.obs['grouped'].median():.1f}")
    print(f"Average UMIs per metacell: {metacells.obs['total_umis'].mean():.1f}")
    
    return fig

In [86]:
# Make directory for outputs if it doesn't exist
output_dir = Path('metacell_outputs')
output_dir.mkdir(exist_ok=True)

# Create visualization
fig = visualize_metacells(metacells, combined_data, seed=42)

# Save plot with higher resolution and proper padding
plt.savefig(output_dir / 'metacell_analysis.pdf', 
            bbox_inches='tight', 
            dpi=300,
            pad_inches=0.1)
plt.close()

set metacells.var[marker_gene]: 6503 true (19.41%) out of 33502 bools


Computing metacell marker-based UMAP...
Finding marker genes...
Computing UMAP using marker genes...
Found 6503 marker genes

Computing standard scanpy UMAP...
Computing UMAP...
normalizing counts per cell
    finished (0:00:00)
extracting highly variable genes
    finished (0:00:00)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)
computing PCA
    with n_comps=50


/Users/guyshani/Documents/PHD/Aim_2/process_sc/.venv_metacells/lib/python3.11/site-packages/scanpy/preprocessing/_pca.py:374: FutureWarning: Argument `use_highly_variable` is deprecated, consider using the mask argument. Use_highly_variable=True can be called through mask_var="highly_variable". Use_highly_variable=False can be called through mask_var=None
  warn(msg, FutureWarning)


    finished (0:00:00)
computing neighbors
    using 'X_pca' with n_pcs = 50
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:00)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm)
    'umap', UMAP parameters (adata.uns) (0:00:00)

Batch Mixing Statistics:
Average batch mixing score: 0.245
Median batch mixing score: 0.256

Metacell Summary Statistics:
Total number of metacells: 576
Average cells per metacell: 75.6
Median cells per metacell: 73.0
Average UMIs per metacell: 176612.7


In [ ]:
# Improved pipeline based on vignette
# with no class structure

# 1. Import libraries and set configurations
import os
import scanpy as sc
import metacells as mc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import scipy.sparse as sp
from pathlib import Path
from math import hypot

# Configure metacells settings
mc.ut.allow_inefficient_layout(False)

# Configure plotting
sb.set_style("white")
plt.rcParams['figure.figsize'] = [10, 10]

# Define global parameters
PROPERLY_SAMPLED_MIN_CELL_TOTAL = 800
PROPERLY_SAMPLED_MAX_CELL_TOTAL = 20000
PROPERLY_SAMPLED_MAX_EXCLUDED_GENES_FRACTION = 0.25
RANDOM_SEED = 42

# Define gene lists for mouse data
EXCLUDED_GENE_NAMES = [
    "Xist", "Malat1",   # Sex-specific genes
    "Neat1"             # Non-coding
]
EXCLUDED_GENE_PATTERNS = ["mt-.*"]  # Mitochondrial genes

LATERAL_GENE_NAMES = [
    "Mki67", "Pcna", "Mcm2", "Mcm3", "Mcm4", "Mcm5", "Mcm6", "Mcm7",
    "Cdc20", "Cdk1", "Top2a", "Ccnb1", "Ccna2", "Aurka", "Birc5",
    "Cdc25c", "Tubb", "Tuba1b"
]
LATERAL_GENE_PATTERNS = ["Rpl.*", "Rps.*"]  # Ribosomal genes

NOISY_GENE_NAMES = [
    "Ccl3", "Ccl4", "Ccl5", "Dusp1", "Fos", "Jun", "Junb",
    "Mt2a", "Hbb", "Tuba1b", "Tubb"
]

def setup_output_dirs(base_dir="metacell_output"):
    """Create output directory structure"""
    base_dir = Path(base_dir)
    dirs = {
        'base': base_dir,
        'preliminary': base_dir / 'preliminary',
        'figures': base_dir / 'preliminary' / 'figures'
    }
    for d in dirs.values():
        d.mkdir(parents=True, exist_ok=True)
    return dirs

def clean_data(adata, output_dirs, doublet_cells=None):
    """Clean the data by removing problematic cells and genes"""
    print("\nCleaning data...")
    print(f"Initial data: {adata.n_obs} cells, {adata.n_vars} genes")
    
    # 1. Mark doublets if provided
    if doublet_cells is not None:
        doublet_cells_mask = pd.Series(False, index=adata.obs_names)
        doublet_cells_mask[doublet_cells] = True
        mc.ut.set_o_data(adata, "doublet_cell", doublet_cells_mask)
    
    # 2. Exclude genes
    print("Excluding genes...")
    mc.pl.exclude_genes(
        adata,
        excluded_gene_names=EXCLUDED_GENE_NAMES,
        excluded_gene_patterns=EXCLUDED_GENE_PATTERNS,
        random_seed=RANDOM_SEED
    )
    
    # 3. Compute and visualize UMI distribution
    total_umis_per_cell = mc.ut.get_o_numpy(adata, "__x__", sum=True)
    plt.figure(figsize=(10, 6))
    sb.histplot(total_umis_per_cell, log_scale=(10, None))
    plt.axvline(PROPERLY_SAMPLED_MIN_CELL_TOTAL, color='green', linestyle='--')
    plt.axvline(PROPERLY_SAMPLED_MAX_CELL_TOTAL, color='red', linestyle='--')
    plt.title('UMI Distribution')
    plt.savefig(output_dirs['figures'] / 'umi_distribution.pdf')
    plt.close()
    
    # 4. Compute excluded gene UMIs
    mc.tl.compute_excluded_gene_umis(adata)
    
    # 5. Exclude cells based on criteria
    print("Excluding cells...")
    mc.pl.exclude_cells(
        adata,
        properly_sampled_min_cell_total=PROPERLY_SAMPLED_MIN_CELL_TOTAL,
        properly_sampled_max_cell_total=PROPERLY_SAMPLED_MAX_CELL_TOTAL,
        properly_sampled_max_excluded_genes_fraction=PROPERLY_SAMPLED_MAX_EXCLUDED_GENES_FRACTION,
        additional_cells_masks=["|doublet_cell"] if doublet_cells is not None else None
    )
    
    # 6. Extract clean data
    clean = mc.pl.extract_clean_data(adata)
    print(f"Clean data: {clean.n_obs} cells, {clean.n_vars} genes")
    
    return clean

def prepare_for_metacells(adata):
    """Prepare the data for metacell computation"""
    print("\nPreparing for metacell computation...")
    
    # 1. Mark lateral genes
    print("Marking lateral genes...")
    mc.pl.mark_lateral_genes(
        adata,
        lateral_gene_names=LATERAL_GENE_NAMES,
        lateral_gene_patterns=LATERAL_GENE_PATTERNS
    )
    
    # 2. Mark noisy genes
    print("Marking noisy genes...")
    mc.pl.mark_noisy_genes(
        adata,
        noisy_gene_names=NOISY_GENE_NAMES
    )
    
    # 3. Set parallel processing
    max_parallel_piles = mc.pl.guess_max_parallel_piles(adata)
    print(f"Using {max_parallel_piles} parallel piles")
    mc.pl.set_max_parallel_piles(max_parallel_piles)
    
    return adata

def compute_metacells(adata):
    """Compute metacells from prepared data"""
    print("\nComputing metacells...")
    
    # 1. Run the divide and conquer pipeline
    mc.pl.divide_and_conquer_pipeline(
        adata,
        random_seed=RANDOM_SEED
    )
    
    # 2. Collect metacells
    metacells = mc.pl.collect_metacells(
        adata,
        random_seed=RANDOM_SEED
    )
    print(f"Created {metacells.n_obs} metacells")
    
    # 3. Compute metrics for MCView
    print("Computing visualization metrics...")
    mc.pl.compute_for_mcview(
        adata=adata,
        gdata=metacells,
        random_seed=RANDOM_SEED
    )
    
    return metacells

def plot_metacell_qc(metacells, output_dirs):
    """Create QC plots for metacells"""
    print("\nCreating QC plots...")
    
    # 1. UMAP plot
    plt.figure(figsize=(12, 12))
    umap_x = mc.ut.get_o_numpy(metacells, "x")
    umap_y = mc.ut.get_o_numpy(metacells, "y")
    umap_edges = sp.coo_matrix(mc.ut.get_oo_proper(metacells, "obs_outgoing_weights"))
    
    plt.scatter(umap_x, umap_y, s=10, alpha=0.7)
    
    # Plot edges between distant metacells
    for source_index, target_index, weight in zip(umap_edges.row, umap_edges.col, umap_edges.data):
        if hypot(umap_x[target_index] - umap_x[source_index], 
                umap_y[target_index] - umap_y[source_index]) >= 4:
            plt.plot([umap_x[source_index], umap_x[target_index]], 
                    [umap_y[source_index], umap_y[target_index]],
                    linewidth=weight * 2, color='indigo', alpha=0.3)
    
    plt.title("Metacells UMAP")
    plt.savefig(output_dirs['figures'] / 'metacells_umap.pdf')
    plt.close()
    
    # 2. Metacell size distribution
    plt.figure(figsize=(8, 6))
    sb.histplot(data=metacells.obs['grouped'])
    plt.title('Metacell Size Distribution')
    plt.xlabel('Cells per Metacell')
    plt.savefig(output_dirs['figures'] / 'metacell_sizes.pdf')
    plt.close()
    
    # Print summary statistics
    print("\nMetacell Summary Statistics:")
    print(f"Total number of metacells: {len(metacells)}")
    print(f"Average cells per metacell: {metacells.obs['grouped'].mean():.1f}")
    print(f"Median cells per metacell: {metacells.obs['grouped'].median():.1f}")
    print(f"Average UMIs per metacell: {metacells.obs['total_umis'].mean():.1f}")

def save_results(adata, metacells, output_dirs, prefix=""):
    """Save the processed data and metacells"""
    print("\nSaving results...")
    
    # Save data
    adata.write(output_dirs['preliminary'] / f"{prefix}cells.h5ad")
    metacells.write(output_dirs['preliminary'] / f"{prefix}metacells.h5ad")
    
    print(f"Results saved to {output_dirs['preliminary']}")

# Example usage in notebook:
"""
# Setup output directories
output_dirs = setup_output_dirs("metacell_output")

# Load data
input_data = sc.read_h5ad("input_data.h5ad")

# Run pipeline steps
clean_data = clean_data(input_data, output_dirs)
prepared_data = prepare_for_metacells(clean_data)
metacells = compute_metacells(prepared_data)

# Create QC plots
plot_metacell_qc(metacells, output_dirs)

# Save results
save_results(prepared_data, metacells, output_dirs, prefix="mouse_")
"""

In [ ]:
os.chdir("/Users/guyshani/Documents/PHD/scenic_test/")

In [4]:
### SCENIC pipeline
import os
import numpy as np
import pandas as pd
import scanpy as sc
import loompy as lp
from MulticoreTSNE import MulticoreTSNE as TSNE

In [ ]:
# set variables for file paths to read from and write to:

# set a working directory
wdir = "/Users/guyshani/Documents/PHD/scenic_test/"
os.chdir( wdir )

# path to unfiltered loom file (this will be created in the optional steps below)
f_loom_path_unfilt = "pbmc10k_unfiltered.loom" # test dataset, n=500 cells

# # path to loom file with basic filtering applied (this will be created in the "initial filtering" step below). Optional.
f_loom_path_scenic = "pbmc10k_filtered_scenic.loom"

# path to anndata object, which will be updated to store Scanpy results as they are generated below
f_anndata_path = "anndata.h5ad"

# path to pyscenic output
f_pyscenic_output = "pyscenic_output.loom"

# loom output, generated from a combination of Scanpy and pySCENIC results:
f_final_loom = 'pbmc10k_scenic_integrated-output.loom'